In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import hstack, kron, eye, csc_matrix, block_diag
#pip install pymatching --upgrade

In [107]:
def repetition_code(n):
    """
    Parity check matrix of a repetition code with length n.
    """
    row_ind, col_ind = zip(*((i, j) for i in range(n) for j in (i, (i+1)%n)))
    #print("row_ind = ", row_ind)
    #print("col_ind = ", col_ind)    
    data = np.ones(2*n, dtype=np.uint8)
    print(csc_matrix((data, (row_ind, col_ind))).toarray())
    return csc_matrix((data, (row_ind, col_ind))).toarray()


In [115]:
from numpy.linalg import inv

a = np.array(repetition_code(3))
#np.matmul(a, )
ainv = inv(a)
synd = [[1],[1],[0]]
print(np.matmul(ainv, synd))

[[1 1 0]
 [0 1 1]
 [1 0 1]]
[[0.]
 [1.]
 [0.]]


In [51]:
def toric_code_x_stabilisers(L):    
    """
    Sparse check matrix for the X stabilisers of a toric code with
    lattice size L, constructed as the hypergraph product of
    two repetition codes.
    """    
    Hr = repetition_code(L)
    H = hstack(
            [kron(Hr, eye(Hr.shape[1])), kron(eye(Hr.shape[0]), Hr.T)],
            dtype=np.uint8
        )
    H.data = H.data % 2
    H.eliminate_zeros()
    print("H = ", H.toarray())
    return csc_matrix(H)



def toric_code_x_logicals(L):
    """
    Sparse binary matrix with each row corresponding to an X logical operator 
    of a toric code with lattice size L. Constructed from the 
    homology groups of the repetition codes using the Kunneth 
    theorem.
    """
    H1 = csc_matrix(([1], ([0],[0])), shape=(1,L), dtype=np.uint8)
    H0 = csc_matrix(np.ones((1, L), dtype=np.uint8))
    x_logicals = block_diag([kron(H1, H0), kron(H0, H1)])
    x_logicals.data = x_logicals.data % 2
    x_logicals.eliminate_zeros()
    return csc_matrix(x_logicals)



def num_decoding_failures_via_physical_frame_changes(H, logicals, error_probability, num_shots):
    matching = Matching.from_check_matrix(H, weights=np.log((1-error_probability)/error_probability))
    num_errors = 0
    for i in range(num_shots):
        noise = (np.random.random(H.shape[1]) < error_probability).astype(np.uint8)
        syndrome = H@noise % 2
        print("syndrome = ", syndrome)
        prediction = matching.decode(syndrome)
        print("prediction = ", prediction)
        predicted_logicals_flipped = logicals@prediction % 2
        actual_logicals_flipped = logicals@noise % 2
        if not np.array_equal(predicted_logicals_flipped, actual_logicals_flipped):
            num_errors += 1
    return num_errors





def num_decoding_failures(H, logicals, p, num_shots):
    matching = Matching.from_check_matrix(H, weights=np.log((1-p)/p), faults_matrix=logicals)
    num_errors = 0
    for i in range(num_shots):
        noise = (np.random.random(H.shape[1]) < p).astype(np.uint8)
        syndrome = H@noise % 2
        print("syndrome = ", syndrome)
        
        predicted_logicals_flipped = matching.decode(syndrome)
        actual_logicals_flipped = logicals@noise % 2
        if not np.array_equal(predicted_logicals_flipped, actual_logicals_flipped):
            num_errors += 1
    return num_errors



def num_decoding_failures_vectorised(H, logicals, error_probability, num_shots):
    matching = Matching.from_check_matrix(H, weights=np.log((1-p)/p), faults_matrix=logicals)
    noise = (np.random.random((num_shots, H.shape[1])) < error_probability).astype(np.uint8)
    shots = (noise @ H.T) % 2
    actual_observables = (noise @ logicals.T) % 2
    predicted_observables = matching.decode_batch(shots)
    num_errors = np.sum(np.any(predicted_observables != actual_observables, axis=1))
    return num_errors





In [56]:
num_shots = 5
Ls = range(2,4,2)
ps = np.linspace(0.1, 0.2, 1)
np.random.seed(2)
log_errors_all_L = []
for L in Ls:
    print("Simulating L={}...".format(L))
    Hx = toric_code_x_stabilisers(L)
    logX = toric_code_x_logicals(L)
    log_errors = []
    for p in ps:
        #num_errors = num_decoding_failures_vectorised(Hx, logX, p, num_shots)
        #num_errors = num_decoding_failures(Hx, logX, p, num_shots)
        num_errors = num_decoding_failures_via_physical_frame_changes(Hx, logX, p, num_shots)
        log_errors.append(num_errors/num_shots)
    log_errors_all_L.append(np.array(log_errors))


Simulating L=2...
H =  [[1 0 1 0 1 1 0 0]
 [0 1 0 1 1 1 0 0]
 [1 0 1 0 0 0 1 1]
 [0 1 0 1 0 0 1 1]]
syndrome =  [0 1 0 1]
prediction =  [0 1 0 0 0 0 0 0]
syndrome =  [0 0 0 0]
prediction =  [0 0 0 0 0 0 0 0]
syndrome =  [1 0 1 0]
prediction =  [1 0 0 0 0 0 0 0]
syndrome =  [0 0 0 0]
prediction =  [0 0 0 0 0 0 0 0]
syndrome =  [0 0 0 0]
prediction =  [0 0 0 0 0 0 0 0]


In [10]:
A = repetition_code(5)
print(A)

  (0, 0)	1
  (4, 0)	1
  (0, 1)	1
  (1, 1)	1
  (1, 2)	1
  (2, 2)	1
  (2, 3)	1
  (3, 3)	1
  (3, 4)	1
  (4, 4)	1


In [11]:
toric_code_x_stabilisers(3)

H =  [[1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1]
 [0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1]]


<9x18 sparse matrix of type '<class 'numpy.uint8'>'
	with 36 stored elements in Compressed Sparse Column format>

In [59]:
import numpy as np
from scipy.sparse import csc_matrix
import pymatching
H = csc_matrix([[1, 1, 0, 0, 0],
                 [0, 1, 1, 0, 0],
                 [0, 0, 1, 1, 0],
                 [0, 0, 0, 1, 1]])
weights = np.array([1, 1, 1, 1, 1])   # Set arbitrary weights for illustration
matching = pymatching.Matching(H, weights=weights)
prediction = matching.decode(np.array([1, 1, 0, 1]))
print("prediction = ", prediction)  # prints: [0 0 1 1 0]
# Optionally, we can return the weight as well:
prediction, solution_weight = matching.decode(np.array([0, 1, 0, 1]), return_weight=True)
print(prediction)  # prints: [0 0 1 1 0]
print(solution_weight)  # prints: 5.0

prediction =  [0 1 0 0 1]
[0 0 1 1 0]
2.0


In [97]:
def num_decoding_failures_noisy_syndromes(H, logicals, p, q, num_shots, repetitions):
    matching = Matching(H, weights=np.log((1-p)/p),
                repetitions=repetitions, timelike_weights=np.log((1-q)/q))
                #repetitions=repetitions, timelike_weights=np.log((1-q)/q), faults_matrix=logicals)
    print("repetitions = ", repetitions)
    num_stabilisers, num_qubits = H.shape
    num_errors = 0
    for i in range(num_shots):
        noise_new = (np.random.rand(num_qubits, repetitions) < p).astype(np.uint8)
        noise_cumulative = (np.cumsum(noise_new, 1) % 2).astype(np.uint8)
        # When calculating noise_cumulative we sum the errors occuring at a single site over 
        # different measurement rounds and then we find the modulo 2 of the total error. 
        print("noise_new = ", noise_new)
        print("noise_cumulative = ", noise_cumulative)
        noise_total = noise_cumulative[:,-1]
        print("noise_total = ", noise_total)
        syndrome = H@noise_cumulative % 2
        print("syndrome = ", syndrome)
        #syndrome_error = (np.random.rand(num_stabilisers, repetitions) < q).astype(np.uint8)
        syndrome_error = (np.random.rand(num_stabilisers, repetitions) < 0).astype(np.uint8)        
        print("syndrome_error = ", syndrome_error)
        
        syndrome_error[:,-1] = 0 # Perfect measurements in last round to ensure even parity
        noisy_syndrome = (syndrome + syndrome_error) % 2
        # Convert to difference syndrome
        noisy_syndrome[:,1:] = (noisy_syndrome[:,1:] - noisy_syndrome[:,0:-1]) % 2
        print("noisy_syndrome = ", noisy_syndrome)
        flipped = matching.decode(noisy_syndrome)
        print("flipped = ", flipped)
        actual_logicals_flipped = noise_total@logicals.T % 2
        if not np.array_equal(flipped, actual_logicals_flipped):
            num_errors += 1
    return num_errors

In [100]:
%%time
from pymatching import Matching
matching=Matching(H)
num_shots = 4
Ls = range(2,4,2)
ps = np.linspace(0.2, 0.4, 1)
print("ps=", ps)
log_errors_all_L = []
for L in Ls:
    print("Simulating L={}...".format(L))
    Hx = toric_code_x_stabilisers(L)
    logX = toric_code_x_logicals(L)
    log_errors = []
    for p in ps:
        nrep = L+1
        num_errors = num_decoding_failures_noisy_syndromes(Hx, logX, p, p, num_shots, nrep)
        log_errors.append(num_errors/num_shots)
    log_errors_all_L.append(np.array(log_errors))


ps= [0.2]
Simulating L=2...
H =  [[1 0 1 0 1 1 0 0]
 [0 1 0 1 1 1 0 0]
 [1 0 1 0 0 0 1 1]
 [0 1 0 1 0 0 1 1]]
repetitions =  3
noise_new =  [[0 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [1 1 0]
 [0 0 0]
 [0 1 1]
 [0 1 0]]
noise_cumulative =  [[0 0 0]
 [0 1 1]
 [1 1 1]
 [0 0 1]
 [1 0 0]
 [0 0 0]
 [0 1 0]
 [0 1 1]]
noise_total =  [0 1 1 1 0 0 0 1]
syndrome =  [[0 1 1]
 [1 1 0]
 [1 1 0]
 [0 1 1]]
syndrome_error =  [[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
noisy_syndrome =  [[0 1 0]
 [1 0 1]
 [1 0 1]
 [0 1 0]]
flipped =  [1 0 0 0 0 0 1 0]
noise_new =  [[0 0 0]
 [0 0 0]
 [1 0 0]
 [0 0 0]
 [0 0 1]
 [0 1 1]
 [0 0 1]
 [0 0 1]]
noise_cumulative =  [[0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 0 1]]
noise_total =  [0 0 1 0 1 0 1 1]
syndrome =  [[1 0 0]
 [0 1 1]
 [1 1 1]
 [0 0 0]]
syndrome_error =  [[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
noisy_syndrome =  [[1 1 0]
 [0 1 0]
 [1 0 0]
 [0 0 0]]
flipped =  [1 0 0 0 1 0 0 0]
noise_new =  [[0 0 1]
 [0 0 0]
 [0 0 1]
 [0 0 0]
 [0 1 0]
 [0 0 0]
 [0 0 

In [ ]:
noise_new =  [[1 0 0]
 [0 1 0]
 [0 0 0]
 [1 1 0]
 [0 1 1]
 [0 0 1]
 [0 0 0]
 [0 0 0]]

noisy_syndrome =  [[1 1 0]
 [1 1 0]
 [1 0 0]
 [1 0 0]]